### Simulation with events

In the [previous](./starting_your_first_simulation.ipynb) notebook we showed you how to start a 3Di simulation. In this notebook
we'll continue with the simulation by adding several events:
- inital waterlevels
- rain
- breaches
- laterals

Let's quickly build a simulation again:

In [ ]:
from datetime import datetime
from getpass import getpass
from threedi_api_client.openapi.api.v3_api import V3Api
from threedi_api_client.api import ThreediApi

In [ ]:
# Provide authentication details
API_HOST = "https://api.3di.live"
PERSONAL_API_KEY = getpass("Personal API token")  # https://management.3di.live/personal_api_keys

config = {
    "THREEDI_API_HOST": API_HOST,
    "THREEDI_API_PERSONAL_API_TOKEN": PERSONAL_API_KEY
}

api_client: V3Api = ThreediApi(config=config)



In [ ]:
# Specify the threedi-model and organisation
organisation_uuid = "b08433fa47c1401eb9cbd4156034c679"
threedimodel = api_client.threedimodels_list(name__icontains='v2_bergermeer').results[0]

# Retrieve first simulation template
simulation_templates = api_client.simulation_templates_list(simulation__threedimodel__id=threedimodel.id)
assert simulation_templates.count > 0, f"No simulation templates found for threedimodel {threedimodel.name}"
simulation_template_id = simulation_templates.results[0].id


In [ ]:
simulation = api_client.simulations_from_template(
    data={
        "template": simulation_template_id,
        "name": "my simulation with events",
        "organisation": organisation_uuid,
        "start_datetime": datetime.now(),
        "duration": 3600  # in seconds, so we simulate for 1 hour
    }
)

print(simulation)

Now that we have created a simulation again, we can start adding some interesting events to
the simulation before starting it.

### Rain

Let's make a time varying rain: every 15 minutes it will decrease in intensity.

In [ ]:
rain = api_client.simulations_events_rain_timeseries_create(
    simulation_pk=simulation.id, data={
        'offset':0,
        'values': [[0, 0.005], [900, 0.002], [1800, 0.001], [2700, 0.0005], [3600, 0]],
        'units': 'm/s'}
)

### Breaches

Breaches are dependent on the selected model. So let's first see if the model has
any breaches configured and if so use the first one.

In [ ]:
potential_breaches = api_client.threedimodels_potentialbreaches_list(threedimodel.id)
print(potential_breaches)

# Let's pick the first potential breach
breach = potential_breaches.results[0]

breach_event = api_client.simulations_events_breaches_create(
    simulation.id, data={
        "potential_breach": breach.id,
        "duration_till_max_depth": 1800,
        "initial_width": 0.5,
        "offset": 1800
    }
)

### Laterals

We'll add a lateral by specifying a connection_node on which we want the lateral to apply.

In [ ]:
lateral = api_client.simulations_events_lateral_constant_create(
    simulation.id, data={
        "offset": 900,
        "connection_node": 4,
        "value": 0.001,
        "duration": 300,
        "units": "m3/s"
    }
)

### Events overview

To get an overview of all our created events on this simulation:

In [ ]:
events = api_client.simulations_events(simulation.id)
print(events)

Now let's start the simulation:

In [ ]:
api_client.simulations_actions_create(simulation.id, data={"name": "start"})

And eventually the simulation will finish!

In [ ]:
api_client.simulations_status_list(simulation.id)